In [37]:
pip install sagemaker==2.127.0

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 655.0/655.0 KB 11.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.127.0-py2.py3-none-any.whl size=890097 sha256=ed690ce4d163f65dce134163497b19fceb855b20e1e613e3476feb10785360c2
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5b/45/09/afaee56ed7903493ef02b1ad4edea087d37d1b3492595dd705
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.125.0
    Uninstalling sagemaker-2.125.0:
      Successfully uninstalled sagemaker-2.125.0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import boto3
import sagemaker

In [1]:
import utils
import preprocessing

Matplotlib is building the font cache; this may take a moment.


In [3]:
train_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/train_data.parquet')
test_data = pd.read_parquet('s3://think-tank-casestudy/raw_data/test_data.parquet')

In [4]:
train_data = train_data[['TRIP_ID','POLYLINE']]
test_data = test_data[['TRIP_ID','POLYLINE']]

In [5]:
train_data['POLYLINE'] = train_data['POLYLINE'].apply(lambda value: json.loads(value))
test_data['POLYLINE'] = test_data['POLYLINE'].apply(lambda value: json.loads(value))

In [3]:
from sagemaker import get_execution_role

In [5]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [15]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0",
    instance_type="ml.t3.medium",
    instance_count=1,
    sagemaker_session=sagemaker_session,
    role=role
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="preprocessing.py",
    inputs=[
        ProcessingInput(source="s3://think-tank-casestudy/raw_data/train_data.parquet", destination="/opt/ml/processing/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train")
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)    

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2023-01-06-17-17-48-702



Job Name:  sagemaker-scikit-learn-2023-01-06-17-17-48-702
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://think-tank-casestudy/raw_data/train_data.parquet', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-206367865313/sagemaker-scikit-learn-2023-01-06-17-17-48-702/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-eu-west-1-206367865313/sagemaker-scikit-learn-2023-01-06-17-17-48-702/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}]
.................

In [7]:
train_data = utils.calculate_POLYLINE_features(train_data)
test_data = utils.calculate_POLYLINE_features(test_data)

In [8]:
train_data = utils.filter_invalid_trips(train_data)
test_data = utils.filter_invalid_trips(test_data)

In [9]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [12]:
test_data = create_timeseries(test_data)

In [ ]:
train_data = create_timeseries(train_data)

In [ ]:
def polyline_to_sequences(data, n_limited):
    start = 2 + n_limited
    sequence_start = data.apply(lambda row: (row.iloc[start:start+n_limited],axis=1)
    sequence_end = data.apply(lambda row: (row.iloc[start+row.N_COORDINATE_POINTS-n_limited:start+row.N_COORDINATE_POINTS])
    return sequence_start, sequence_end